In [1]:
!pip install pandas plotly

In [2]:
import pandas as pd

df0 = pd.read_csv('data/0_preprocessed.csv')
df0

,timestamp,duration,song,artist,album,song_artist
0,7/5/2019 3:00,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,7/5/2019 3:01,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,7/5/2019 3:05,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,7/5/2019 3:08,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,7/5/2019 3:12,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,9/4/2024 13:36,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,9/4/2024 14:35,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,9/5/2024 13:50,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,9/5/2024 16:08,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [3]:
cur_song, cur_duration = '', 0
replayed = {} # key: song, value: [a,b,c] a 10+ b 30+ c 60+

for i, row in df0.iterrows():
  if cur_song != row['song']:
    a,b,c = replayed.get(cur_song, [0,0,0])
    if cur_duration < 30:
      pass
    else:
      if cur_duration < 45:
        a += 1
      elif cur_duration < 60:
        b += 1
      else:
        c += 1
      replayed[cur_song] = [a,b,c]
    cur_duration = 0

  cur_song = row['song']
  cur_duration += row['duration']

len(replayed)

36

In [30]:
df9_js = pd.DataFrame.from_dict(replayed, orient='index', columns=['mins_30_45', 'mins_45_60', 'mins_60_plus'])
df9_js = df9_js.reset_index().rename(columns={'index': 'song'})
df9_js

,song,mins_30_45,mins_45_60,mins_60_plus
0,Delicate,0,0,1
1,Everything Has Changed,2,0,4
2,One Call Away,2,1,2
3,Sparks Fly,10,7,6
4,As Long As You Love Me,3,0,0
5,Enchanted,15,8,12
6,Love Story,1,0,0
7,Into You,1,0,1
8,Come Back...Be Here,1,0,0
9,Treacherous,3,1,0


In [28]:
def scale(x):
  if x == 0:
    x = 0
  elif 1 <= x <= 2:
    x = 5
  elif 3 <= x <= 5:
    x = 6
  elif 6 <= x <= 10:
    x = 7
  elif 11 <= x <= 15:
    x = 8
  else:
    x = 9
  return x

In [31]:
df9_js['mins_30_45'] = df9_js['mins_30_45'].apply(scale)
df9_js['mins_45_60'] = df9_js['mins_45_60'].apply(scale)
df9_js['mins_60_plus'] = df9_js['mins_60_plus'].apply(scale)
df9_js.head(2)

,song,mins_30_45,mins_45_60,mins_60_plus
0,Delicate,0,0,5
1,Everything Has Changed,5,0,6


In [26]:
df9_js.to_csv('9_wind_rose_replayer_js.csv', index=False)

In [22]:
df9 = pd.DataFrame(replayed, index=['30-45 mins','45-60 mins','60+ mins']).transpose()
df9 = df9.reset_index().melt(id_vars='index', var_name='duration', value_name='times')
df9 = df9.rename(columns={'index': 'song'})
df9['scales'] = df9['times'].apply(scale)
df9

,song,duration,times,scales
0,Delicate,30-45 mins,0,0
1,Everything Has Changed,30-45 mins,2,5
2,One Call Away,30-45 mins,2,5
3,Sparks Fly,30-45 mins,10,7
4,As Long As You Love Me,30-45 mins,3,6
...,...,...,...,...
103,Âm Thầm Bên Em,60+ mins,2,5
104,2AM,60+ mins,0,0
105,Suýt Nữa Thì...,60+ mins,6,7
106,Em Gái Mưa,60+ mins,1,5


In [23]:
import plotly.express as px

hover_data = {
    'scales': False,
    'times': True,
    'duration': True,
    'song': True
}

fig = px.bar_polar(df9,
                   r="scales",
                   theta="song",
                   color="duration",
                   template="plotly_dark",
                  #  range_r=[0,20],
                  #  log_r=True,
                   color_discrete_sequence= px.colors.sequential.Plasma_r,
                   hover_data=hover_data,
                  #  showticklabels=False
                   )

fig.update_layout(
    polar=dict(radialaxis=dict(showticklabels=False, ticks='')),
    showlegend=False)
fig.show()

In [24]:
df9.to_csv('9_wind_rose_replayer.csv', index=False)

In [25]:
px.colors.sequential.Plasma_r

['#f0f921',
 '#fdca26',
 '#fb9f3a',
 '#ed7953',
 '#d8576b',
 '#bd3786',
 '#9c179e',
 '#7201a8',
 '#46039f',
 '#0d0887']